In [ ]:
%cd /content/drive/My\ Drive/Adversarial\ training/

/content/drive/My Drive/Adversarial training


#Training 

In [ ]:
!python Train_MNIST.py --attacktrain "cw" --epsilon 0.4

epoch    train_err    test_err    adv_err
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
0.000000	0.392233	0.037700	0.214700
1.000000	0.170983	0.024200	0.120900
2.000000	0.117817	0.021900	0.092000
3.000000	0.093033	0.020000	0.083000
4.000000	0.077933	0.018000	0.071400
5.000000	0.066733	0.015900	0.063000
6.000000	0.059550	0.012700	0.055300
7.000000	0.054283	0.013300	0.054900
8.000000	0.049850	0.012200	0.053200
9.000000	0.046650	0.013100	0.051800
10.000000	0.043433	0.012200	0.048800
11.000000	0.041883	0.011400	0.045700
12.000000	0.038233	0.012300	0.046000
13.000000	0.036467	0.011100	0.044500
14.000000	0.035300	0.010900	0.043400
15.000000	0.033100	0.011100	0.037700
16.000000	0.031433	0.010600	0.037200
17.000000	0.029617	0.009600	0.041900
18.000000	0.028783	0.010100	0.037700
19.000000	0.028050	0.011500	0.034800
20.000000	0.026833	0.009700

#Test

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
torch.manual_seed(0)
mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)    


def pgd_linf(model, X, y, epsilon=0.1, alpha=0.01, num_iter=20, randomize=False):
    """ Construct FGSM adversarial examples on the examples X"""
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)
        
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()
def epoch_adversarial(loader, model, attack, opt=None, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y, **kwargs)
        yp = model(X+delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)




def epoch(loader, model, opt=None):
    """Standard training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [ ]:
model_cnn_robust = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), nn.ReLU(),
                          nn.Linear(100, 10)).to(device)
model_cnn_robust.load_state_dict(torch.load("./Trained_models/MNIST/MNIST_e04_cw.pt"))
clean_err,clean_loss=epoch(test_loader,model_cnn_robust)
print("clean error",clean_err)
adv_err, adv_loss = epoch_adversarial(test_loader, model_cnn_robust, pgd_linf,epsilon=0.3, alpha=0.015,randomize=False,num_iter=20)
print("PGD-20",adv_err)
adv_err, adv_loss = epoch_adversarial(test_loader, model_cnn_robust, pgd_linf,epsilon=0.3, alpha=0.015,randomize=False,num_iter=100)
print("PGD-100",adv_err)